**AI & Machine Learning (KAN-CINTO4003U) - Copenhagen Business School | Spring 2025**

***


# Part II: BERT

Please see the description of the assignment in the README file (section 2) <br>
**Guide notebook**: [guides/bert_guide.ipynb](guides/bert_guide.ipynb)


***

<br>

* Note that you should report results using a classification report. 

* Also, remember to include some reflections on your results: how do they compare with the results from Part I, BoW? Are there any hyperparameters that are particularly important?

* You should follow the steps given in the `bert_guide` notebook

* **Optionally**, you can fine-tune a pre-trained BERT model to classify news articles as is done in [guides/bert_guide_finetuning.ipybb](guides/bert_guide_finetuning.ipybb), the same task as in part 1. As this requires more computational resources, this part is optional. If you do decide to complete this part, you will need to use a GPU (e.g., Google Colab) to train the model. (For reference, training on a 2020 Macbook Pro with 16GB RAM and a M1 chip results in an out-of-memory error). Therefore, we suggest that you use Google Colab or another cloud-based service with a GPU. You can easily upload the `bert_guide_finetuning.ipynb` notebook to Google Colab and run it there.

<br>

***

In [11]:
# imports for the project

from datasets import load_dataset, DatasetDict
from transformers import pipeline
import numpy as np
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

### 1. Load the data

We can load this data directly from [Hugging Face Datasets](https://huggingface.co/docs/datasets/) - The HuggingFace Hub- into a Pandas DataFrame. Pretty neat!

**Note**: This cell will download the dataset and keep it in memory. If you run this cell multiple times, it will download the dataset multiple times.

You are welcome to increase the `frac` parameter to load more data.

In [12]:
ag_news_train = load_dataset("fancyzhx/ag_news", split="train[:2%]")  # % of the training data
ag_news_test = load_dataset("fancyzhx/ag_news", split="test[:20%]")  # % of test data

ag_news = DatasetDict({
    "train": ag_news_train,
    "test": ag_news_test
})

ag_news

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1520
    })
})

In [13]:
embedder = pipeline(
    model="answerdotai/ModernBERT-base",      # model used for embedding
    tokenizer="answerdotai/ModernBERT-base",  # tokenizer used for embedding
    task="feature-extraction",                # feature extraction task (returns embeddings)
    device=0                                  # use GPU 0 if available
)

Device set to use cpu


In [14]:
def get_embeddings(data):
    """ Extract the [CLS] embedding for each text. """
    embeddings = embedder(data["text"])  # Full token embeddings
    cls_embeddings = [e[0][0] for e in embeddings]  # Extract first token ([CLS])
    return {"embeddings": cls_embeddings}

ag_news = ag_news.map(get_embeddings, batched=True, batch_size=8)


Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

In [15]:
ag_news

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'embeddings'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['text', 'label', 'embeddings'],
        num_rows: 1520
    })
})

In [16]:

X_train = np.array(ag_news["train"]["embeddings"])  # Feature embeddings
y_train = np.array(ag_news["train"]["label"])       # Labels

X_test = np.array(ag_news["test"]["embeddings"])
y_test = np.array(ag_news["test"]["label"])

# Check shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (2400, 768), y_train shape: (2400,)
X_test shape: (1520, 768), y_test shape: (1520,)


In [17]:
# Define the parameter grid
param_grid = {
    'C': [0.05, 0.1, 0.5, 1.0],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'class_weight': [None, 'balanced']
}

# Create a grid search object
grid_search = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=1000),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
print("Best parameters:", grid_search.best_params_)

# Use the best model
best_lr_clf = grid_search.best_estimator_

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best parameters: {'C': 0.5, 'class_weight': None, 'penalty': 'l1', 'solver': 'liblinear'}


In [18]:
y_pred_train = best_lr_clf.predict(X_train)

print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90       568
           1       0.93      0.97      0.95       468
           2       0.91      0.85      0.88       508
           3       0.92      0.94      0.93       856

    accuracy                           0.92      2400
   macro avg       0.92      0.91      0.91      2400
weighted avg       0.92      0.92      0.92      2400



In [19]:
y_pred = best_lr_clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       383
           1       0.92      0.95      0.94       404
           2       0.80      0.70      0.75       339
           3       0.80      0.85      0.82       394

    accuracy                           0.84      1520
   macro avg       0.84      0.83      0.83      1520
weighted avg       0.84      0.84      0.84      1520



## Reflections on Results and Hyperparameter Choices

My implementation of the BERT-based classification model achieved 84% accuracy on the test set, compared to 77% in the bert_guide notebook.

The size of both the train and test data sets were increased for better results by giving the model more to work with.

**Hyperparameter Optimization:**
   - Implemented GridSearchCV to find optimal hyperparameters
   - C=0.5 with L1 regularization provided the best balance of performance and generalization

### Conclusion

The improved performance demonstrates the impact of proper hyperparameter tuning and an increase in data when working with BERT embeddings. 